In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('AirPort').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/08 15:43:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
rate_df = spark.read.options(sep=';', header=True, inferSchema=True).csv("./BooksRating-CSV/Book-Ratings.csv")
rate_avg_df = rate_df.groupBy('isbn').avg('rate')
rate_df = rate_df.join(rate_avg_df, on="isbn", how="left_outer")
rate_df.show(5)

+----------+------+----+------------------+
|      isbn|userid|rate|         avg(rate)|
+----------+------+----+------------------+
|0425190048|261274|  10|               5.0|
|0767915054|211919|   0| 2.480769230769231|
|2080674722|276733|   0|3.6666666666666665|
|034545104X|276725|   0| 2.933333333333333|
|0451204530|261272|   8|3.1743119266055047|
+----------+------+----+------------------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import collect_list

grouped_rates_df = rate_df.groupBy('isbn').agg(collect_list('rate').alias('rates_list'))

joined_df = rate_df.join(grouped_rates_df, 'isbn', 'left_outer')

In [13]:
joined_df.show()

+----------+------+----+------------------+--------------------+
|      isbn|userid|rate|         avg(rate)|          rates_list|
+----------+------+----+------------------+--------------------+
|0060517794|276747|   9|3.6363636363636362|[7, 10, 0, 8, 0, ...|
|0373241801|106936|   0|               0.0|                 [0]|
|0380006081|106936|   0|               0.0|                 [0]|
|0380718421|261275|   7|1.7619047619047619|[0, 8, 0, 0, 0, 0...|
|0425190048|261274|  10|               5.0|             [0, 10]|
|0553576348|261289|   7|             2.625|[0, 0, 9, 0, 0, 0...|
|0767915054|211919|   0| 2.480769230769231|[0, 0, 9, 0, 5, 4...|
|0786013990|276746|   0|3.2857142857142856|[0, 8, 7, 0, 0, 8...|
|0787602035|211919|   6|               6.0|                 [6]|
|0803493762|211919|   0|               0.0|              [0, 0]|
|0805062262|211919|   0|               0.0|           [0, 0, 0]|
|0805241221|211919|   0|               0.0|     [0, 0, 0, 0, 0]|
|0811802981|211919|  10| 

In [40]:
from pyspark.sql.functions import explode
exploded_df = joined_df.withColumn('rate_explode', explode('rates_list')).drop('rates_list')
exploded_df.orderBy('isbn').show()

+-----------+------+----+---------+------------+
|       isbn|userid|rate|avg(rate)|rate_explode|
+-----------+------+----+---------+------------+
| 0330299891| 89192|   0|      3.0|           0|
| 0330299891| 89192|   0|      3.0|           6|
| 0330299891|181817|   6|      3.0|           0|
| 0330299891|181817|   6|      3.0|           6|
| 0375404120| 89192|   3|      1.5|           3|
| 0375404120| 89192|   3|      1.5|           0|
| 0375404120|266865|   0|      1.5|           3|
| 0375404120|266865|   0|      1.5|           0|
| 0586045007| 89192|   0|      0.0|           0|
| 9022906116| 11676|   7|      3.5|           7|
| 9022906116| 11676|   7|      3.5|           0|
| 9022906116| 89192|   0|      3.5|           7|
| 9022906116| 89192|   0|      3.5|           0|
| 9032803328| 89192|   0|      0.0|           0|
| 9044922564| 89192|   0|      0.0|           0|
| 9044922572| 89192|   0|      0.0|           0|
| 9044922718| 89192|   0|      0.0|           0|
| 9044923161| 89192|

In [41]:
rate_df.count()

1149780

In [42]:
joined_df.count()

1149780

In [43]:
exploded_df.count()

56552234